In [137]:
#importamos librerías necesarias y generamos dataframe a partir de los datos
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import os
import requests
import json
import plotly.express as px
from plotly.subplots import make_subplots
from datetime import datetime
from meteostat import Point, Daily
import patoolib


#generamos dataframe
gun_weather = pd.read_csv('C:\coderhouse\coderhouse\proyecto_final\gun_violence_full.csv')

In [138]:
gun_weather_ill = gun_weather[gun_weather.city_or_county == "Rockford"]
gun_weather_alg = gun_weather_ill[['date', 'incident_amount']]
gun_weather_day = gun_weather_alg.groupby('date').sum()


In [139]:
def setweather():
    #time period
    start = datetime(2014, 1, 1)
    end = datetime(2016, 12, 31)
    
    #location point
    location = Point(42.2615, -89.0808)

    # Get daily data 
    data = Daily(location, start, end)
    data = data.fetch()
    return data

In [164]:
w_data = setweather()
w_data.index = w_data.index.astype("datetime64[ns]")
gun_weather_day.index = gun_weather_day.index.astype("datetime64[ns]")
gun_alg = w_data.join(gun_weather_day)
gun_alg['incident_amount'].fillna('False', inplace=True)
gun_alg.loc[gun_alg["incident_amount"] != "False", "incident_amount"] = True
gun_alg = gun_alg.drop(['tsun', 'wpgt', 'wdir'], axis=1)
gun_alg['pres'] = gun_alg['pres'].fillna(gun_alg['pres'].median())
gun_alg['day'] = gun_alg.index.day
gun_alg['month'] = gun_alg.index.month
gun_alg['day_of_the_week'] = gun_alg.index.dayofweek
gun_alg.rename(columns = {'incident_amount':'incident'}, inplace = True)


In [166]:
gun_alg.head()

,tavg,tmin,tmax,prcp,snow,wspd,pres,incident,day,month,day_of_the_week
time,,,,,,,,,,,
2014-01-01,-12.4,-13.8,-8.8,5.8,230.0,22.7,1026.7,True,1,1,2
2014-01-02,-12.1,-23.8,-10.5,0.0,280.0,19.4,1027.7,False,2,1,3
2014-01-03,-18.7,-27.7,-7.1,0.0,280.0,21.2,1025.4,True,3,1,4
2014-01-04,-5.2,-7.1,-1.6,4.8,280.0,18.4,1012.5,False,4,1,5
2014-01-05,-8.8,-21.6,-6.6,0.0,330.0,30.2,1020.2,False,5,1,6
